In [1]:
# Import libraries
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import os
import yaml
from haversine import haversine
from itertools import product
from math import sqrt
from math import floor, ceil
import datetime
from collections import Counter

In [2]:
# Change working directory
os.chdir('../../')

In [3]:
# Load configuration file
dict_config = yaml.safe_load(open("src/config.yml", "r"))
print(dict_config)

{'default': {'grid_size': '1km', 'stores_filename': 'processing_store-coordinates-and-sales-area-by-chain.xlsx', 'stores_ms_filename': '.confidential_market-share-by-store.xlsx', 'sales_filename': 'turnover-by-chain-2021.xlsx', 'alpha_filename': 'modelling_alpha-parameters.xlsx', 'x_offset': 550, 'y_offset': 550, 'last_closing_date': '2021-12-31', 'new_stores_date': '2021-01-01', 'distance_threshold_km': 100, 'alpha_select': 'alpha_est_avg', 'remove_other_stores': True, 'evaluate_ms_on_store_level': True, 'use_average_alpha': True}, 'config_name': {'config_var': ''}}


### Import and process data

In [4]:
# Load and inspect population data
df_pop = pd.read_excel(str('./data/processed/')+'processing_population-by-'+dict_config['default']['grid_size']+'-grid.xlsx')
display(df_pop.head())
display(df_pop.info())

,grid_id,grid_size,x_coord_lks94,y_coord_lks94,population,mean_age,latitude_y,longitude_x
0,1x1kmX30800Y61332,1x1km,308550,6133750,3,34.000000,55.301836,20.984556
1,1x1kmX30800Y61342,1x1km,308550,6134750,6,40.833333,55.310807,20.983875
2,1x1kmX30900Y61332,1x1km,309550,6133750,1409,45.965224,55.302223,21.000283
3,1x1kmX30900Y61342,1x1km,309550,6134750,281,41.412811,55.311195,20.999605
4,1x1kmX31000Y61332,1x1km,310550,6133750,25,39.720000,55.302609,21.016009


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34664 entries, 0 to 34663
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   grid_id        34664 non-null  object 
 1   grid_size      34664 non-null  object 
 2   x_coord_lks94  34664 non-null  int64  
 3   y_coord_lks94  34664 non-null  int64  
 4   population     34664 non-null  int64  
 5   mean_age       34664 non-null  float64
 6   latitude_y     34664 non-null  float64
 7   longitude_x    34664 non-null  float64
dtypes: float64(3), int64(3), object(2)
memory usage: 2.1+ MB


None

In [5]:
# Load and inspect stores data
df_stores = pd.read_excel(str('./data/processed/')+dict_config['default']['stores_filename'])
display(df_stores.head())
display(df_stores.info())

,store_no,address,chain_format,opening_date,closing_date,longitude_x,latitude_y,sales_area_m2,chain,x_coord_lks94,y_coord_lks94
0,89,Taikos prospektas 100,AIBE,1900-01-01,2049-12-31,21.171887,55.668067,1553,AIBE,322098.686,6173998.559
1,115,Vilniaus g. 11,AIBE,1900-01-01,2049-12-31,24.047287,54.398666,75,AIBE,503070.564,6029086.000
2,116,V. Sladkeviciaus g. 2,AIBE,2020-07-31,2049-12-31,24.184648,54.162399,300,AIBE,512058.840,6002807.050
3,131,Taikos 20,AIBE,1900-01-01,2049-12-31,24.412276,56.060760,700,AIBE,525677.451,6214161.754
4,132,Salduvs g.,AIBE,1900-01-01,2049-12-31,23.421890,55.920864,65,AIBE,463863.902,6198662.972


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1688 entries, 0 to 1687
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   store_no       1688 non-null   object 
 1   address        1688 non-null   object 
 2   chain_format   1688 non-null   object 
 3   opening_date   1688 non-null   object 
 4   closing_date   1688 non-null   object 
 5   longitude_x    1688 non-null   float64
 6   latitude_y     1688 non-null   float64
 7   sales_area_m2  1688 non-null   int64  
 8   chain          1688 non-null   object 
 9   x_coord_lks94  1688 non-null   float64
 10  y_coord_lks94  1688 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 145.2+ KB


None

In [6]:
# Remove unidentified chains
if dict_config['default']['remove_other_stores'] == True:
    df_stores = df_stores[df_stores['chain']!='OTHER']

In [7]:
# Load Alpha parameter
df_alpha = pd.read_excel(str('./data/processed/')+dict_config['default']['alpha_filename'])
df_alpha = df_alpha[["chain", dict_config['default']['alpha_select']]]
df_alpha.rename(columns={dict_config['default']['alpha_select']: "alpha"}, inplace=True)

display(df_alpha)

# Add alpha parameter to df_stores
df_stores = pd.merge(df_stores, df_alpha, how="left", on="chain")
display(df_stores.head())

,chain,alpha
0,AIBE,0.999894
1,RIMI,0.946585
2,MAXIMA,1.022871
3,LIDL,1.052330
4,NORFA,0.948460
5,IKI,0.993894


,store_no,address,chain_format,opening_date,closing_date,longitude_x,latitude_y,sales_area_m2,chain,x_coord_lks94,y_coord_lks94,alpha
0,89,Taikos prospektas 100,AIBE,1900-01-01,2049-12-31,21.171887,55.668067,1553,AIBE,322098.686,6173998.559,0.999894
1,115,Vilniaus g. 11,AIBE,1900-01-01,2049-12-31,24.047287,54.398666,75,AIBE,503070.564,6029086.000,0.999894
2,116,V. Sladkeviciaus g. 2,AIBE,2020-07-31,2049-12-31,24.184648,54.162399,300,AIBE,512058.840,6002807.050,0.999894
3,131,Taikos 20,AIBE,1900-01-01,2049-12-31,24.412276,56.060760,700,AIBE,525677.451,6214161.754,0.999894
4,132,Salduvs g.,AIBE,1900-01-01,2049-12-31,23.421890,55.920864,65,AIBE,463863.902,6198662.972,0.999894


In [8]:
# Create df for candidate locations
df_candidates = df_stores[(df_stores['chain']=="IKI") & (df_stores['opening_date'] >= dict_config['default']['new_stores_date'])]
display('DF shape: ' + str(df_candidates.shape))

'DF shape: (10, 12)'

In [10]:
# Create df for competition stores
df_competition = df_stores[(df_stores['chain']!="IKI") & (df_stores['opening_date'] < dict_config['default']['new_stores_date'])]
print(str('All competitors stores'))
display(df_competition.head())
display(df_competition.shape)

# Create df for existing own stores
df_existing_own = df_stores[(df_stores['chain']=="IKI") & (df_stores['opening_date'] < dict_config['default']['new_stores_date'])]
print(str('All existing own stores'))
display(df_existing_own.shape)

All competitors stores


,store_no,address,chain_format,opening_date,closing_date,longitude_x,latitude_y,sales_area_m2,chain,x_coord_lks94,y_coord_lks94,alpha
0,89,Taikos prospektas 100,AIBE,1900-01-01,2049-12-31,21.171887,55.668067,1553,AIBE,322098.686,6173998.559,0.999894
1,115,Vilniaus g. 11,AIBE,1900-01-01,2049-12-31,24.047287,54.398666,75,AIBE,503070.564,6029086.000,0.999894
2,116,V. Sladkeviciaus g. 2,AIBE,2020-07-31,2049-12-31,24.184648,54.162399,300,AIBE,512058.840,6002807.050,0.999894
3,131,Taikos 20,AIBE,1900-01-01,2049-12-31,24.412276,56.060760,700,AIBE,525677.451,6214161.754,0.999894
4,132,Salduvs g.,AIBE,1900-01-01,2049-12-31,23.421890,55.920864,65,AIBE,463863.902,6198662.972,0.999894


(1282, 12)

All existing own stores


(225, 12)

### Modelling

#### Create input variables

In [11]:
# Set maximum catchment area.
# This value is based on the analysis of customer choice rules in step 2.2
threshold = 3

In [12]:
# Create lists for candidate stores, exisitng own stores and existing competitors' stores
eo_store_no = [row['store_no'] for ind,row in df_existing_own.iterrows()]
ec_store_no = [row['store_no'] for ind,row in df_competition.iterrows()]
c_store_no = [row['store_no'] for ind,row in df_candidates.iterrows()]

# Dictionary for all stores
a_store_no, a_sales_area, a_coordinates, a_alpha = gp.multidict({
        row['store_no']: [row['sales_area_m2'], (float(row['latitude_y']), float(row['longitude_x'])), float(row['alpha'])] for ind,row in df_stores[(df_stores['chain']=="IKI") | (df_stores['opening_date'] < dict_config['default']['new_stores_date'])].iterrows()
    })

# Dictionary for demand points
d_grid_id, d_demand, d_coordinates = gp.multidict({
        row['grid_id']: [row['population'], (float(row['latitude_y']), float(row['longitude_x']))] for ind,row in  df_pop.iterrows()
    })

# Check total number of stores
print('Existing own stores + existing competitors\' stores + candidate locations ')
display(len(eo_store_no) + len(ec_store_no) + len(c_store_no))
print('Number of all stores')
display(len(a_store_no))

Existing own stores + existing competitors' stores + candidate locations 


1517

Number of all stores


1517

In [13]:
# Dictionary of valid demand-facility pairings with attractiveness value.
# Valid demand-facility pairs are defined by catchment threshold.
valid_pairings, attractiveness = gp.multidict({(facility, demand): (a_sales_area[facility]**a_alpha[facility])/haversine(a_coordinates[facility], d_coordinates[demand])
                                               for facility in a_store_no
                                               for demand in d_grid_id 
                                               if haversine(a_coordinates[facility], d_coordinates[demand]) < threshold})

#### Check if there are demand points that are only connected to candidate facilities
This is done in order to avoid infeasibility due to division by 0.  
Relevant when small threshold values are used.

In [14]:
# Find all facility-demand pairings for demand points that are connected to any of the candidate facilities
c_grid_pairings = [(demand, facility) 
                   for (facility, demand) in valid_pairings 
                   if demand  in [demand for (facility, demand) in valid_pairings.select(c_store_no,'*')]]

In [15]:
# Check the number of facilities that a demand point is connected to.
grid_facility_count = Counter(elem[0] for elem in c_grid_pairings)
# Get demand points that are only connected to candidate facilities.
grids_with_one_facility = [k for k, v in grid_facility_count.items() if v == 1]
print(grids_with_one_facility)

['1x1kmX48700Y60802']


In [16]:
# Get facility-demand pairings that should be removed to avoid infeasibility of the model.
# Model becomes infeasible because with only candidate facility connected to the demand point, denominator may become 0.
pairings_to_remove = [(facility, demand) for facility, demand in valid_pairings.select('*',grids_with_one_facility)]
print(pairings_to_remove)

[('I1/00168', '1x1kmX48700Y60802')]


In [17]:
# Remove facility-demand pairing from valid_pairings dictionary
[valid_pairings.pop(valid_pairings.index((facility, demand))) for facility, demand in pairings_to_remove]

[('I1/00168', '1x1kmX48700Y60802')]

In [18]:
# Remove faiclity-demand pairing from attractiveness dictionary
[attractiveness.pop((facility, demand)) for facility, demand in pairings_to_remove]

[401.012570712505]

In [19]:
# Generate a list of valid grid_ids
d_valid_grid_id = set([demand for (facility, demand) in valid_pairings])

In [20]:
# Check how many demand points are not covered by any of the facilities
display('Number of demand points: ' + str(len(d_grid_id)))
display('Number of covered demand points: ' + str(len(d_valid_grid_id)))

'Number of demand points: 34664'

'Number of covered demand points: 10167'

In [21]:
# Get boundaries for attractiveness
min_attract = floor(min([sum(attractiveness.select('*',demand)) for demand in d_valid_grid_id]))
display(min_attract)
max_attract = ceil(max([sum(attractiveness.select('*',demand)) for demand in d_valid_grid_id]))
display(max_attract)

5

127049

#### Model formulation: new and existing facilities

In [23]:
# Run for loop to generate models for different number of facilities to build.
# Based on the results, candidate facilities are ranked and compared with actual rankings.

# Two sets of models are built: one where new and existing own facilities are considered
# when searching for optimal location and second where only new facilities are considered. 
# This is done in order to check for cannibalization effect.

for tf in [True, False]:
    
    if tf == True:
        print('##### New and existing own facilities included #####')
    else:
        print('##### New facilities only #####')
    
    for run in range(1,len(c_store_no), 1):
        start = datetime.datetime.now()

        max_facilities = run
        display(run)

        # Logical condition to check if the model should take into account existing own facilities or not
        include_existing = tf

        # MIP  model formulation
        m = gp.Model('flp-cjh')

        # Select facilities from candidate locations
        select = m.addVars(c_store_no, 
                           vtype=GRB.BINARY, 
                           name='select')

        # Set constraint for maximum number of facilities to open
        m.addConstr(select.sum() <= max_facilities, 
                    name="facility_limit")

        # Set NonConvex parameter to 2 to account for non-linearity of the model
        m.setParam("NonConvex", 2)

        # Create numerator variable
        zn = m.addVars(d_valid_grid_id, 
                       vtype=GRB.CONTINUOUS, 
                       name='numerator')

        if include_existing == True:
            m.addConstrs((zn[demand] == (gp.quicksum([attractiveness.sum(facility,demand) * select[facility] 
                                                      for facility in c_store_no])
                                         + attractiveness.sum(eo_store_no, demand)) # Existing own facilities included
                          for demand in d_valid_grid_id),
                         name="set_numerator")
        else:
            m.addConstrs((zn[demand] == (gp.quicksum([attractiveness.sum(facility,demand) * select[facility] 
                                                      for facility in c_store_no])) 
                          for demand in d_valid_grid_id),
                         name="set_numerator")

        # Create denominator variable
        zd = m.addVars(d_valid_grid_id, 
                       vtype=GRB.CONTINUOUS, 
                       name='denominator', 
                       lb = min_attract, 
                       ub = max_attract)
        m.addConstrs((zd[demand] == (gp.quicksum([attractiveness.sum(facility,demand) * select[facility] 
                                                  for facility in c_store_no])
                                     + attractiveness.sum(eo_store_no, demand)
                                     + attractiveness.sum(ec_store_no, demand)) 
                      for demand in d_valid_grid_id),
                     name="set_denominator")

        # Create auxiliary variable for linear reformulation
        z = m.addVars(d_valid_grid_id, 
                      vtype=GRB.CONTINUOUS, 
                      name='z', 
                      lb = 1/max_attract, 
                      ub = 1/min_attract)
        m.addConstrs((zd[demand] * z[demand] == 1
                      for demand in d_valid_grid_id),
                     name="auxiliary_variable_constraint")
        m.update()

        # Formulate objective function
        m.setObjective(gp.quicksum(d_demand[demand] * zn[demand] * z[demand]
                                   for demand in d_valid_grid_id),
                   GRB.MAXIMIZE)
        m.optimize()

        for facility in select.keys():
            if (abs(select[facility].x) > 1e-6):
                print(f"\n Build facility at location {facility}.")

        finish = datetime.datetime.now()
        display(finish)
        print('Runing time: ' + str(finish-start))

##### New and existing own facilities included #####


1

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20335 rows, 30511 columns and 20906 nonzeros
Model fingerprint: 0x11124005
Model has 10167 quadratic objective terms
Model has 10167 quadratic constraints
Variable types: 30501 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e+00, 3e+04]
  Bounds range     [8e-06, 1e+05]
  RHS range        [1e+00, 9e+04]
  QRHS range       [1e+00, 1e+00]
Presolve removed 20192 rows and 30291 columns
Presolve time: 0.14s
Presolved: 570 rows, 292 columns, 1444 nonzeros
Presolved model has 142 bilinear constraint(s)
Variable types: 285 continuous, 7 integer (7 binary)

Root relaxation: objective 4.115865e+05, 383 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node   

datetime.datetime(2023, 1, 5, 9, 38, 59, 47992)

Runing time: 0:02:30.306794


...

Runing time: 0:02:06.543596


9

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20335 rows, 30511 columns and 20906 nonzeros
Model fingerprint: 0x4d278e7e
Model has 10167 quadratic objective terms
Model has 10167 quadratic constraints
Variable types: 30501 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e+00, 3e+04]
  Bounds range     [8e-06, 1e+05]
  RHS range        [6e+00, 9e+04]
  QRHS range       [1e+00, 1e+00]
Presolve removed 20192 rows and 30290 columns
Presolve time: 0.09s
Presolved: 570 rows, 293 columns, 1375 nonzeros
Presolved model has 142 bilinear constraint(s)
Variable types: 285 continuous, 8 integer (8 binary)

Root relaxation: objective 2.392294e+04, 267 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node   

datetime.datetime(2023, 1, 5, 10, 17, 43, 303375)

Runing time: 0:02:02.894633
